In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url="http://api.scraperapi.com", #scrapeapi
            params={
                'api_key': 'e3458ff579ef5c1369f185af0ceaecfb',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v3.csv')
            line_count+=1

    

In [14]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("temp_44_v3.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v3.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url_v3.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('temp_45_v3.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [15]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/somerset-west-business-park/somerset-west/western-cape/16198/110170443
{'name': '2 Bedroom Apartment / Flat for Sale in Somerset West Business Park', 'price': 'R 1 200 000', 'province': 'western-cape', 'city': 'somerset-west', 'address': '\n2 Bedroom Apartment / Flat for Sale in Somerset West Business Park\n', 'Listing Number': '110170443', 'Type of Property': 'Apartment / Flat', 'Street Address': ' 14 Niblick Way, , Somerset West Business Park', 'Listing Date': '06 July 2021', 'Bedrooms': '2', 'Bedroom 1': 'Tiled Floors, Built-in Cupboards', 'Bedroom 2': 'Tiled Floors, Built-in Cupboards', 'Bathrooms': '2', 'Bathroom 1': 'Tiled Floors, En suite, Shower, Toilet and Basin', 'Bathroom 2': 'Tiled Floors, Bath, Toilet and Basin', 'Kitchen': 'Stove (Oven & Hob), Extractor Fan, Washing Machine Connection, Tiled Floors, Open Plan', 'Lounge': 'Tiled Floors, Balcony, Open Plan', 'Parking': 'Single, Underground', 'Garden': 'Communal', 'Pool': 'Fenced, Communal Pool',

{'name': '3 Bedroom Townhouse for Sale in Seemeeu Park', 'price': 'R 995 000', 'province': 'western-cape', 'city': 'mossel-bay', 'address': 'Seemeeu Park, Mossel Bay', 'Listing Number': '110233570', 'Type of Property': 'Townhouse', 'Description': 'Semi Detached, Duplex', 'Lifestyle': 'Coastal/Beach, Complex', 'Listing Date': '26 July 2021', 'Erf Size': '152 m²', 'Floor Size': '124 m²', 'Levies': 'R 530', 'Rates and Taxes': 'R 880', 'Pets Allowed': 'Yes', 'Coverage': '81 %', 'Bedrooms': '3', 'Bedroom 1': 'Tiled Floors, Built-in Cupboards', 'Bedroom 2': 'Tiled Floors, Built-in Cupboards', 'Bedroom 3': 'Tiled Floors, Balcony, Blinds, Built-in Cupboards', 'Bathrooms': '2', 'Bathroom 1': 'Tiled Floors, Main en Suite, Shower, Toilet and Basin', 'Bathroom 2': 'Tiled Floors, Full', 'Kitchen': 'Open Plan, Extractor Fan, Tiled Floors, Built-in Cupboards, Under Counter Oven, Hob', 'Family/TV Room': 'Tiled Floors, Open Plan', 'Garage': 'Electric Door, Single, Tip up', 'Parking': 'Single', 'Garden'

==>Property link:  /for-sale/oudorp/klerksdorp/north-west/6430/109719512


KeyboardInterrupt: 